In [1]:
import mne
import os
import numpy as np 
import pandas as pd
import pywt

In [20]:
import pywt
import mne
import os
import numpy as np
import pandas as pd

# Wavelet Transform Features
def wavelet_features(signal, wavelet='db4', level=4):
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    # Extract statistics from coefficients at each level (e.g., variance, energy, etc.)
    features = []
    for coeff in coeffs:
        features.append(np.mean(coeff))        # Mean of the coefficients
        features.append(np.std(coeff))         # Standard deviation
        features.append(np.var(coeff))         # Variance
        features.append(np.sum(np.abs(coeff))) # Energy (sum of absolute values)
    return features

# Hjorth Parameters
def hjorth_parameters(signal):
    activity = np.var(signal)
    mobility = np.sqrt(np.var(np.diff(signal)) / activity)
    complexity = np.sqrt(np.var(np.diff(np.diff(signal))) / np.var(np.diff(signal)))
    return activity, mobility, complexity

# Bandpower Ratios
def bandpower_ratio(fft, freqs):
    delta_power = np.sum(np.abs(fft[(freqs >= 0.5) & (freqs < 4)])**2)
    theta_power = np.sum(np.abs(fft[(freqs >= 4) & (freqs < 8)])**2)
    alpha_power = np.sum(np.abs(fft[(freqs >= 8) & (freqs < 13)])**2)
    beta_power = np.sum(np.abs(fft[(freqs >= 13) & (freqs < 30)])**2)

    theta_alpha_ratio = theta_power / alpha_power if alpha_power != 0 else 0
    delta_theta_ratio = delta_power / theta_power if theta_power != 0 else 0

    return delta_power, theta_power, alpha_power, beta_power, theta_alpha_ratio, delta_theta_ratio

# Consolidated Feature Extraction Function (with Duration)
def process_edf_features(edf_file_path, output_csv_path):
    # Read file EDF using MNE
    raw = mne.io.read_raw_edf(edf_file_path, preload=True)
    # raw.filter(l_freq=0.5, h_freq=50.0)
    raw.filter(1, 50.0)
    first_4_channels = raw.get_data()[0::2, :]

    # Compute average signal
    sum_signal = np.sum(first_4_channels, axis=0)
    avg_signal = sum_signal / first_4_channels.shape[0]

    # Frequency domain features (FFT)
    fft = np.fft.fft(avg_signal)
    freqs = np.fft.fftfreq(len(avg_signal), d=1/raw.info['sfreq'])

    # Power in different frequency bands and bandpower ratios
    delta_power, theta_power, alpha_power, beta_power, theta_alpha_ratio, delta_theta_ratio = bandpower_ratio(fft, freqs)

    # Hjorth parameters
    activity, mobility, complexity = hjorth_parameters(avg_signal)

    # Time-domain features
    variance = np.var(avg_signal)
    power = np.mean(avg_signal ** 2)
    median = np.median(avg_signal)
    signal_range = np.max(avg_signal) - np.min(avg_signal)

    # Wavelet features
    wavelet_feats = wavelet_features(avg_signal, wavelet='db4', level=4)

    # Calculate duration of the signal
    duration = raw.times[-1]  # Duration in seconds

    # Prepare data for saving (keep 'status' as the last item)
    data = {
        'variance': [variance],
        'power': [power],
        'median': [median],
        'range': [signal_range],
        'delta_power': [delta_power],
        'theta_power': [theta_power],
        'alpha_power': [alpha_power],
        'beta_power': [beta_power],
        'theta_alpha_ratio': [theta_alpha_ratio],
        'delta_theta_ratio': [delta_theta_ratio],
        'activity': [activity],
        'mobility': [mobility],
        'complexity': [complexity],
        'duration': [duration]  # Added duration
    }

    # Add wavelet features
    for i, wf in enumerate(wavelet_feats):
        data[f'wavelet_feat_{i+1}'] = [wf]

    # Add 'status' at the end as the label column
    data['status'] = ['epilepsy']  # Update this based on patient status

    # Convert to DataFrame and save
    df = pd.DataFrame(data)

    # Append or write data to CSV
    if os.path.exists(output_csv_path):
        df.to_csv(output_csv_path, mode='a', header=False, index=False)
    else:
        df.to_csv(output_csv_path, mode='w', header=True, index=False)


In [22]:
path = "process_epilepsy_11to13"
# path = "yes_under_1m"
output_dir = "finaltrain22222.csv"

# Loop through all EDF files in the directory
for filename in os.listdir(path):
    if filename.endswith('.edf'):
        input_dir = os.path.join(path, filename)
        
        # Process the file and extract features
        process_edf_features(input_dir, output_dir)
        
    print(f"Processed: {input_dir}")

Extracting EDF parameters from d:\FGW\EDF\v2.0.1_new\process_epilepsy_11to13\aaaaaanr_s007_t004.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 198911  =      0.000 ...   776.996 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 845 samples (3.301 s)

Processed: process_epilepsy_11to13\aaaaaanr_s007_t004.edf
Extracting EDF parameters from d:\FGW\EDF\v2.0.1_new\process_epilepsy_11to13\aaaaabdn_s005_t008.edf...
EDF file detected
Setting channel i

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 825 samples (3.300 s)

Processed: process_epilepsy_11to13\aaaaabdn_s005_t008.edf
Extracting EDF parameters from d:\FGW\EDF\v2.0.1_new\process_epilepsy_11to13\aaaaabhz_s003_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 261999  =      0.000 ...   654.997 secs...
Filtering raw data in 1 contiguous segment


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 1321 samples (3.303 s)

Processed: process_epilepsy_11to13\aaaaabhz_s003_t001.edf
Extracting EDF parameters from d:\FGW\EDF\v2.0.1_new\process_epilepsy_11to13\aaaaabhz_s005_t002.edf...


...

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


Processed: process_epilepsy_11to13\aaaaaovm_s007_t002.edf
Extracting EDF parameters from d:\FGW\EDF\v2.0.1_new\process_epilepsy_11to13\aaaaapkm_s003_t007.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 169983  =      0.000 ...   663.996 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 845 samples (3.301 s)

Processed: process_epilepsy_11to13\aaaaapkm_s003_t007.edf


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
